<a href="https://colab.research.google.com/github/volosciucedmond/Proiect-LAB-MEDII-VIZUALE-2022/blob/main/Tem%C4%83_compara%C8%9Bie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%html
<p style = "font-size:300%; text-align:center; color:#336600"> <b> Traffic Flow - Linear Regression and Random Forest</b></p>

Problema de flux de trafic poate fi abordată folosind algoritmi de regresie liniară și random forest, bazându-se pe un set de date care conține următoarele câmpuri: momentul zilei, ziua săptămânii, vremea, tipul drumului (autostradă, stradă din oraș) și nivelul de flux de trafic (redus, mediu, mare). Scopul este de a construi un model care să poată prezice nivelul de flux de trafic într-un anumit moment al zilei, într-o anumită zi a săptămânii, în funcție de condițiile meteorologice și de tipul drumului. Pentru a rezolva această problemă, se vor utiliza algoritmi de regresie liniară și random forest pe baza unui set de date care conține informații despre momentul zilei, ziua săptămânii, vremea, tipul drumului și nivelul de flux de trafic.



Data setul:


*   time_of_day: momentul zilei
*   day_of_week: ziua saptamanii
*   weather: vremea
*   road_type: tipul drumului
*   traffic_flow: nivelul fluxului de trafic

Importăm librăriile necesare

In [ ]:
import numpy as np # algebra liniară
from sklearn.model_selection import train_test_split # imparte un set de date in doua subseturi: unul pentru antrenare si unul pentru testare
from sklearn.linear_model import LinearRegression # regresie liniara
import pandas as pd # procesarea de date
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import preprocessing
from google.colab import files # pentru încărcare sau descărcare de fișiere din calculator în Google Colab

Facem upload la setul de date

In [ ]:
uploaded = files.upload()

Saving tf2.csv to tf2.csv


Citim si afișăm informații despre setul de date

In [ ]:
import io # oferă instrumente pentru manipularea intrării și ieșrii de date în și din aplciație
df = pd.read_csv(io.BytesIO(uploaded['tf2.csv'])) # citirea setului de date
# df = pd.read_csv("drive/My Drive/traffic.csv") # uplaod de pe Google Drive
df.info() # oferă informații despre setul de date
df.describe() # descrierea setului de date
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   time_of_day   60 non-null     object
 1   day_of_week   60 non-null     object
 2   weather       60 non-null     object
 3   road_type     60 non-null     object
 4   traffic_flow  60 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB


,time_of_day,day_of_week,weather,road_type,traffic_flow
0,Morning,Monday,Clear,Highway,Heavy
1,Afternoon,Tuesday,Partly cloudy,City street,Moderate
2,Evening,Wednesday,Rainy,Highway,Light
3,Night,Thursday,Snowy,City street,Heavy
4,Morning,Friday,Clear,Highway,Moderate


Vom impărți setul de date în variabile independente și variabile dependente

In [ ]:
X = df[['time_of_day', 'day_of_week', 'weather', 'road_type']]
y = df['traffic_flow']

Acum vom antrena și testa. Astfel, alocăm 80% din setul de date de mai sus pentru antrenare, iar restul de 20% pentru testare.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
# train_test_split imparte setul de date în date de antrenare și date de testare
# test_size reprezintă mărimea setului de date de testare
# random_state setează aleatorietatea

Vom face encoding la setul de date (object -> int)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in df.columns:
  if df[col].dtype == 'object':
    le.fit(df[col])
    df[col] = le.transform(df[col])
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   time_of_day   60 non-null     int64
 1   day_of_week   60 non-null     int64
 2   weather       60 non-null     int64
 3   road_type     60 non-null     int64
 4   traffic_flow  60 non-null     int64
dtypes: int64(5)
memory usage: 2.5 KB


,time_of_day,day_of_week,weather,road_type,traffic_flow
0,2,1,0,1,0
1,0,5,1,0,2
2,1,6,2,1,1
3,3,4,3,0,0
4,2,0,0,1,2
5,0,2,1,0,1
6,1,3,2,1,0
7,2,1,0,0,2
8,0,5,3,1,0
9,1,6,2,0,1


Astfel, datele din dataset au fost convertite astfel:


*   Morning = 2, afternoon = 0, evening = 1, night = 3
*   monday = 1, tuesday = 5, wednesday = 6, thursday = 4, friday = 0, saturday = 2, sunday = 3
*   clear = 0, partly cloduy = 1, rainy = 2, snowy = 3
*   highway = 1, city street = 0
*   heavy = 0, moderate = 2, light = 1





Vom impărți setul de date în variabile independente și variabile dependente

In [ ]:
X = df[['time_of_day', 'day_of_week', 'weather', 'road_type']]
y = df['traffic_flow']

Acum vom antrena și testa. Astfel, alocăm 80% din setul de date de mai sus pentru antrenare, iar restul de 20% pentru testare.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
# train_test_split imparte setul de date în date de antrenare și date de testare
# test_size reprezintă mărimea setului de date de testare
# random_state setează aleatorietatea

Acum vom aplica cei doi algoritmi pentru setul nostru de date (Funcția fit este folosită pentru finalizarea procesului de antrenare)

In [ ]:
# regresia liniară
modelLR = LinearRegression()
modelLR.fit(X_train, y_train)
y_predLR = modelLR.predict(X_test)
print(y_predLR)
predictions = modelLR.predict(X_test)

# random forest
clf = RandomForestClassifier(n_estimators = 100)
clf.fit(X_train,y_train)
y_predRF = clf.predict(X_test)
print(y_predRF)

# afisam acuratetea

print("Accuracy LR:", modelLR.score(X_test, y_test))
print("Accuracy RF:", metrics.accuracy_score(y_test, y_predRF))




[ 1.61870845 -0.00931156 -0.02503439  0.57436568  0.54292003  1.63443128
  1.38291253  0.72051033  0.5873176   1.93791502  1.87779462  1.83339704]
[2 0 0 1 1 2 1 1 0 2 2 2]
Accuracy LR: 0.8322546718013372
Accuracy RF: 1.0


Observăm că dacă aveam random state-ul setat pe 4, acuratețea folosind random forest este de 100% pentru un set de date cu 60 de linii, pe când la regresia liniară este de doar 83%.

Acum vom realiza o predictie pentru un nou input: Afternoon, Wednesday, Snowy, City street.

In [ ]:
import warnings

try:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        print("LR: ", modelLR.predict([[0, 6, 3, 0]]))
        print("RF: ", clf.predict([[0, 6, 3, 0]]))
except:
    pass


LR:  [0.36729077]
RF:  [0]


Observăm că ambii algoritmi tind spre același rezultat (dacă rotunjim spre 0 primul rezultat)

Vom mai face un test pe un set de date nou: evening, sunday, clear, highway

In [ ]:
import warnings

try:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        print("LR: ", modelLR.predict([[1, 3, 0, 1]]))
        print("RF: ", clf.predict([[1, 3, 0, 1]]))
except:
    pass


LR:  [1.66784152]
RF:  [2]


Din nou, dacă e să rotunjim primul rezultat in sus, obținem acealși rezultat

Astfel, putem vedea că și dacă acuratețea celor doi algoritmi diferă 17%, acestea duc la același rezultat.